# Key Workshop 2023

## In this workshop

- importing data and pre-processing for analysis
- key techniques
- inverted tables
- batch processing


---

## Importing data
- Not specified in problem statement
- In the session: can use `]Get` (v18.2)

In [3]:
]get ./order_data.csv
3↑order_data

#.order_data

┌──┬───────────────────┬─────────┬─────┬───────┬──────────┐
│id│timestamp │city │state│payment│category │
├──┼───────────────────┼─────────┼─────┼───────┼──────────┤
│1 │2017-10-02 10:56:33│sao paulo│SP │18.12 │housewares│
├──┼───────────────────┼─────────┼─────┼───────┼──────────┤
│2 │2018-07-24 20:41:37│barreiras│BA │141.46 │perfumery │
└──┴───────────────────┴─────────┴─────┴───────┴──────────┘

In [51]:
⎕PW←3000

In [52]:
1↓30↑order_data[;5]

18.12 141.46 179.12 72.2 28.62 175.26 75.16 35.95 161.42 259.06 114.13 50.13 32.7 54.36 146.45 1376.45 43.09 29.75 62.15 118.86 88.9 17.28 187.57 104.11 125.43 227.68 60.69 192.68 63.34

---

### `⎕CSV`
How can we import numeric data as numbers?

In [8]:
1↓30↑5(⌷⍤1)⎕CSV 'order_data.csv' ⍬ 4

18.12 141.46 179.12 72.2 28.62 175.26 75.16 35.95 161.42 259.06 114.13 50.13 32.7 54.36 146.45 1376.45 43.09 29.75 62.15 1
 18.86 88.9 17.28 187.57 104.11 125.43 227.68 60.69 192.68 63.34

**Question:** When is `⎕CSV path ⍬ 4` dangerous?

- Numeric "codes" e.g. US zip codes
- Telephone numbers
- Hexadecimal `12E056`

Safer to use full description.  
`⎕CSV path ⍬ (2 1 1 1 2 1)`  

In [9]:
]repr 1⌷order_data

'id' 'timestamp' 'city' 'state' 'payment' 'category'

**Exercise**

Given a known `col_spec` mapping matrix:

```
path←'/path/to/order_data.csv'
col_spec←⍪'payment' 'id' 'city' 'state' 'category' 'timestamp'
col_spec,← 2         2    1      1       1          1
```

We want to write a function with this syntax:

```
(data header)←col_spec ReadOrderData path
```

Reading the header:

```APL
tn←'order_data.csv' ⎕ntie 0
(_ header)←⎕CSV ⎕OPT'Records' 1⊢tn ⍬ 1 1
⎕nuntie tn
```

In [ ]:
tn←'order_data.csv' ⎕ntie 0
(_ header)←⎕CSV ⎕OPT'Records' 1⊢tn ⍬ 1 1
col_types ← ((⊢/col_spec),1)[(⊣/col_spec)⍳header]   ⍝ Mapping between actual column order and column type specification
data←⎕CSV tn ⍬ col_types
⎕nuntie tn

---

### Date times
Were provided as `YYYY-MM-DD hh:mm:ss`.

Some extracted year and month as text. Some extracted integer numbers.

**Exercise:** Convert `YYYY-MM-DD hh:mm:ss` into `¯1↓⎕TS`-style numeric timestamp.

- For a simple timestamp vector, return a simple numeric vector.
- For a nested list of timestamp vectors, return a nested list of numeric vectors.

In [56]:
Timestamp2TS←{⊃⍣(1=≡⍵)⊢2⊃¨'- :'∘⎕VFI¨⊆⍵}

In [57]:
(data cols)←⎕CSV 'order_data.csv' ⍬ 4 1
⍴Timestamp2TS ⊃data[;2]
⍴Timestamp2TS 5↑data[;2]

6

5

In [63]:
{⊃⍣(1=≡⍵)(⍎¨∊∘⎕D⊆⊢)¨⊆⍵} ⊃data[;2]
{1=≡⍵:⍎¨(⍵∊⎕D)⊆⍵ ⋄ ∇¨⍵} 3↑data[;2]
⍝ Possible future extension?
⍝ (⍎¨∊∘⎕D⊆⊢)⍥1

2017 10 2 10 56 33

┌──────────────────┬──────────────────┬────────────────┐
│2017 10 2 10 56 33│2018 7 24 20 41 37│2018 8 8 8 38 49│
└──────────────────┴──────────────────┴────────────────┘

In [74]:
Timestamp2TS←{↓⎕CSV((,¨'- :')⎕R','⊆⍵)'N'4}
⍴Timestamp2TS ⊃data[;2]
⍴Timestamp2TS 5↑data[;2]

1

5

#### What is the purpose of these expressions? What are their edge cases?

```
a) ⊂⍣(1=≡⍵)⊢⍵
b) ↓↑⍵
c) (1+80=⎕DR ⍵)⊃⍵(⊂⍵)
```

In [84]:
{,⍤⊂⍣(1≥|≡⍵)⊢⍵}'SP' 'GT'
{,⍤⊂⍣(1=≡,⍵)⊢⍵}'SP'
{(1+1=≡,⍵)⊃⍵(,⊂⍵)}'SP'

{⊂⍣(1=≡⍵)⊢⍵}'S'

┌──┬──┐
│SP│GT│
└──┴──┘

┌──┐
│SP│
└──┘

┌──┐
│SP│
└──┘

S

In [79]:
↓↑'SP' 'RDJ' 'BR'

┌───┬───┬───┐
│SP │RDJ│BR │
└───┴───┴───┘

In [83]:
{(1+80=⎕DR ⍵)⊃⍵(⊂⍵)} 'İstanbul'

İstanbul

The subtle fiddliness of scalar/1-elem vector/vector/1-row matrix is a bit pedantic. Just remember to be consistent and document expected arguments and results.

---

## Aggregating data
The main problem:

- select relevant columns (keys)
- apply aggregate function on keys
- ensure correct ordering and shape of result

Payment per state

Write a function `PaymentPerState` which:
- accepts a nested vector of character vectors of state codes
- returns the total payment in each given state across the whole dataset.

```
      PaymentPerState 'GO' 'TO' 'SC'
319766.98 58068.18 579297.8
```

**Exercise:** Spot the errors  
This code is problematic. What issues can you spot?

```APL
PPS←{
     sp ← (⊂cols⍳'state' 'payment') (⌷⍤1) data     
     sp ⌿⍨← (⊣/sp)∊⍵
     (⊣/sp) {+/⍵}⌸ (⊢/sp)
 }
```

1. Valid state missing in data set
1. Data with keys in order found in data, rather than order of `⍵`

Several ways to mitigate each.

**Exercise:** Fix the issues with `PPS`

```APL
PPS←{
     sp ← (⊂cols⍳'state' 'payment') (⌷⍤1) data     
     sp ⌿⍨← (⊣/sp)∊⍵
     (⊣/sp) {+/⍵}⌸ (⊢/sp)
 }
```

1. Prepend data and keys with fill and dictionary
1. Use a lookup `⍺⍳⍵` after `⌸`

```APL
PPS←{
     sp ← (⊂cols⍳'state' 'payment') (⌷⍤1) data     
     sp ⌿⍨← (⊣/sp)∊⍵
     (⍵,⊣/sp) {+/⍵}⌸ (-⍵+⍥≢sp)↑(⊢/sp)
 }
```

```APL
PPS←{
     sp ← (⊂cols⍳'state' 'payment') (⌷⍤1) data     
     sp ⌿⍨← (⊣/sp)∊⍵
     (gs tot) ←↓⍉ (⊣/sp) {⍺,+/⍵}⌸ (⊢/sp)
     tot[gs⍳⍵]
 }
```

**Note:** Performance of `('state' From cols) ∊ states`

- Filtering may be improved by pre-computing numeric "IDs", but lookup still required
- Lookup may be improved by using inverted tables
- Ultimately a storage / database issue
- Admittedly this problem is more a "quick analysis" than suited to industrial-strength processing

```
]runtime -c "s ∊ 'SP' 'BA'" "InStates'SP' 'BA'" "sid∊(s⍳'SP' 'BA')" "{(≢⍵)≥sm⍳⍨↑⍵}'SP' 'BA'"
```

---

### Grouping by date-time / computing intervals
Problems `PaymentPerMonth` and `PaymentPerQuarter` allowed for different approaches:

- Modular approach / code re-use
- Directly compute quarter intervals

**Exercise 1:** Given `ppm←PaymentPerMonth states`, how can we compute `ppq←PaymentPerQuarter`?

**Exercise 2:** Given numeric months `1...12`, return corresponding quarters `1...4`

**Correction:** note about result orientation (transpose) of `ppm/ppq`.

In [85]:
⍝ +/¨(3/⍳4)⊂ppm
⍝ +/{((≢⍵),4 3)⍴⍵}ppm
⍝ ]runtime -c "+/¨(3/⍳4)⊆ppm" "{+/4 3(⍴⍤1)⍵}ppm" "{+/((¯1↓⍴⍵),4 3)⍴⍵}ppm"

In [86]:
⍝ +\1=3|m
⍝ ⌈3÷⍨
⍝ +⌿4 7 10∘.≤m
⍝ 4 7 10⍸m

**Exercise:** Write the function `PaymentsBetween` which:

- `⍵:` takes as argument a nested vector of character vectors `(1=≢⍴⍵)∧(2=≡⍵)` of dates from oldest (1st element) to most recent (last element)
- `←:` returns a vector of length (`¯1+≢⍵`) of total payments between dates specified